In [1]:
using BSON: @load
using Flux
using Flux.Optimise
using Flux.Optimise: update!
using Flux.Data: DataLoader
using ImageFiltering
using Images
using ImageIO
using MLDatasets: FashionMNIST
using LinearAlgebra
using MLDatasets
using Plots
using Zygote
using FFTW
using Distributions
using SparseArrays
using JLD
using StatsBase
using LaTeXStrings

In [2]:
function sample_fourier(m, n)

    F = dct(diagm(ones(n)),2)
    index = zeros(m)
    StatsBase.self_avoid_sample!(Vector(1:1:n),index)
    return F[Int.(index),:]
end

sample_fourier (generic function with 1 method)

Let $A_d$ be a $n\times k$ matrix and $A$ be a $n\times n$ unitary matrix.  The incoherence of the subspace $\textit{Range}(A_d)$ with respect to the unitary matrix $A$ equals
$$ \sup_{x\in\mathcal{R}(A_d)\cap \mathcal{S}^{n-1}} \|A x\|_\infty = \max_{i \in [n]}\left\{\sup_{u\in\mathbb{R}^{k}} \langle a_i, A_d u\rangle \text{ s.t. } \|A_du\|_2 = 1 \right\}$$
where $a_i$ is the $i$-th row of $A$. Consider a (thin)-QR decompition of $A_d = QR$. Then,
$$\sup_{u\in\mathbb{R}^{k}} \langle a_i, A_d u\rangle \text{ s.t. } \|A_du\|_2 = 1 \Leftrightarrow  \sup_{u\in\mathbb{R}^{k}} \langle Q^\top a_i, R u\rangle \text{ s.t. } \|Ru\|_2 = 1.$$
Consider a change of variable of $u = R^{-1} v$, for some $v \in \mathbb{R}^k$. It is easy to see that this supremum value is $\|Q^\top a_i\|_2$. Thus,
$$ \sup_{x\in\mathcal{R}(A_d)\cap \mathcal{S}^{n-1}} \|A x\|_\infty = \max_{i \in [n]} \|Q^\top a_i\|_2.$$


We note that $\sup_{x\in\mathcal{R}(A_d)\cap \mathcal{S}^{n-1}} \|A x\|_\infty$ only provides an upper bound to the incoherence of the GNN w.r.t $A$ given by
$$ \sup_{x\in\mathcal{R}(G)\cap \mathcal{S}^{n-1}}\|A x\|_\infty.$$


In [11]:
function subspace_incoherence(F, A)
    m, _ = size(A)
    Q = Matrix(qr(A).Q)
    temp = Q'*F'
    return maximum(sqrt.(sum(temp.*temp, dims = 1)))

end

subspace_incoherence (generic function with 1 method)

In [4]:
# function load_model(load_dir::String, epoch::Int)
#     print("Loading model...")
#     @load joinpath(load_dir, "model-$epoch.bson") encoder_μ encoder_logvar decoder
#     println("Done")
#     return encoder_μ, encoder_logvar, decoder
# end

# function load_model_identity(load_dir::String, epoch::Int)
#     print("Loading model...")
#     @load joinpath(load_dir, "model-$epoch.bson") encoder_μ encoder_logvar decoder decoder_last
#     println("Done")
#     return encoder_μ, encoder_logvar, decoder, decoder_last
# end

load_model (generic function with 1 method)

In [15]:

function relative_error(z₀, z_est)
    return(norm(z₀ - z_est, 2)/ norm(z₀, 2))
end  

relative_error (generic function with 1 method)

In [5]:
function perturbed_weights(x, n)
    A = x
    for i in 1:n-1
        A = hcat(A, a + randn(length(x)))
    end
    _, s, _ = svd(A)
    A = A/s[1];
    return A
end

perturbed_weights (generic function with 1 method)

Consider the compressed sensing problem of recovering $x\in\mathbb{R}^n$ from noisy measurements of the form

$$y = A x_{0} + \epsilon, $$

where $\epsilon\in\mathbb{R}^n$ is noise and $A$ is the compressive sensing matrix. We assume the unknown signal $x$ lives in the range of known generative model $G:\mathbb{R}^k \rightarrow \mathbb{R}^n$, i.e. $x_{0} = G(z_0)$ for some $z_0 \in \mathbb{R}^k$. We assume the generative model $G$ is  fully-connected feedforward network of the form 

$$ G(x) = A_d\sigma(A_{d-1} \cdots \sigma(A_1 z)\cdots),$$

where $A_i \in \mathbb{R}^{n_i \times n_{i-1}}$ is the weight matrix and $\sigma(\cdot)$ is the activation function. We
determine the conditions (on $A, G, x_{0}$, \etc) under which it is possible to (approximately) recover $x_{0}$ from noisy linear measurements $y$ by (approximately) solving an optimization problem of the form

$$\argmin_{z \in \mathbb{R}^{k}} \|b - A G(z) \|_{2}. $$

Although this optimzation problem is non-convex, it has been shown that gradient descent and other descent-type alogorithm can provably converge to the global optima. 

In [13]:
function estimated_code(opt, G, y, A, init; max_iter, tolerance, out_toggle = 0)
    z_est = init
    θ = Flux.params(z_est)
    iter = 1
    ∇f = 1.0
    while ∇f >= tolerance && iter <= max_iter
        grads = gradient(() -> loss(z_est, y, G, A), θ)
        update!(opt, z_est, grads[z_est])
        ∇f = norm(grads[z_est], 2)
        iter += 1
        if out_toggle != 0 && iter % out_toggle == 0
            println("====> In Gradient: Iteration: $iter gradient norm: $∇f")
        end
    end
    return z_est
end


loss (generic function with 1 method)

In [14]:
function loss(z, y, G, A)
    return norm(A*G(z) -y, 2)^2
end

loss (generic function with 1 method)

In [12]:
function get_β_α(F, A, B, n)
    # generate "uniform" gapped incoherence
    β_list = 0:.001:1
    α_list = []
    for β in β_list
        push!(α_list, subspace_incoherence(F, β * A + (1-β)*B) )
        
    end

    α_list_ideal = LinRange(α_list[1],α_list[end], n)
    index_list = []
    for α_ideal in α_list_ideal
        temp = 0
        index = 0
        for i in 1:length(α_list)
            if abs(temp - α_ideal) > abs(α_list[i] - α_ideal)
                temp = α_list[i]
                index = i
            end
        end
        push!(index_list, index)
    end
    β_list = β_list[index_list]
    α_list = α_list[index_list]

    return β_list, α_list
end

get_β_α (generic function with 1 method)

In [ ]:
cs = palette([:red,  :orange, :green, :blue, :Indigo], 1000)

In [ ]:
println("All function imported")